# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7cd0f97790>
├── 'a' --> tensor([[-1.4916, -0.4511,  0.0633],
│                   [ 0.2273,  0.1149, -1.1401]])
└── 'x' --> <FastTreeValue 0x7f7cd0f97d60>
    └── 'c' --> tensor([[ 1.2364, -1.2485,  1.6212, -0.5242],
                        [ 1.2824,  1.5641,  0.6374,  1.2199],
                        [ 0.6198,  0.5993, -0.3168, -1.2198]])

In [4]:
t.a

tensor([[-1.4916, -0.4511,  0.0633],
        [ 0.2273,  0.1149, -1.1401]])

In [5]:
%timeit t.a

62.4 ns ± 0.0942 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7cd0f97790>
├── 'a' --> tensor([[ 0.8631, -0.5453,  0.3890],
│                   [-0.6123, -0.1058,  0.0302]])
└── 'x' --> <FastTreeValue 0x7f7cd0f97d60>
    └── 'c' --> tensor([[ 1.2364, -1.2485,  1.6212, -0.5242],
                        [ 1.2824,  1.5641,  0.6374,  1.2199],
                        [ 0.6198,  0.5993, -0.3168, -1.2198]])

In [7]:
%timeit t.a = new_value

63.7 ns ± 0.0971 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.4916, -0.4511,  0.0633],
               [ 0.2273,  0.1149, -1.1401]]),
    x: Batch(
           c: tensor([[ 1.2364, -1.2485,  1.6212, -0.5242],
                      [ 1.2824,  1.5641,  0.6374,  1.2199],
                      [ 0.6198,  0.5993, -0.3168, -1.2198]]),
       ),
)

In [10]:
b.a

tensor([[-1.4916, -0.4511,  0.0633],
        [ 0.2273,  0.1149, -1.1401]])

In [11]:
%timeit b.a

56.8 ns ± 0.0173 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1565,  1.1823,  0.0819],
               [ 0.3541,  1.0959, -1.0624]]),
    x: Batch(
           c: tensor([[ 1.2364, -1.2485,  1.6212, -0.5242],
                      [ 1.2824,  1.5641,  0.6374,  1.2199],
                      [ 0.6198,  0.5993, -0.3168, -1.2198]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.175 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

813 ns ± 0.431 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 33.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 779 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7bf30a98e0>
├── 'a' --> tensor([[[-1.4916, -0.4511,  0.0633],
│                    [ 0.2273,  0.1149, -1.1401]],
│           
│                   [[-1.4916, -0.4511,  0.0633],
│                    [ 0.2273,  0.1149, -1.1401]],
│           
│                   [[-1.4916, -0.4511,  0.0633],
│                    [ 0.2273,  0.1149, -1.1401]],
│           
│                   [[-1.4916, -0.4511,  0.0633],
│                    [ 0.2273,  0.1149, -1.1401]],
│           
│                   [[-1.4916, -0.4511,  0.0633],
│                    [ 0.2273,  0.1149, -1.1401]],
│           
│                   [[-1.4916, -0.4511,  0.0633],
│                    [ 0.2273,  0.1149, -1.1401]],
│           
│                   [[-1.4916, -0.4511,  0.0633],
│                    [ 0.2273,  0.1149, -1.1401]],
│           
│                   [[-1.4916, -0.4511,  0.0633],
│                    [ 0.2273,  0.1149, -1.1401]]])
└── 'x' --> <FastTreeValue 0x7f7bf30a9310>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 94.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7cd0fa1790>
├── 'a' --> tensor([[-1.4916, -0.4511,  0.0633],
│                   [ 0.2273,  0.1149, -1.1401],
│                   [-1.4916, -0.4511,  0.0633],
│                   [ 0.2273,  0.1149, -1.1401],
│                   [-1.4916, -0.4511,  0.0633],
│                   [ 0.2273,  0.1149, -1.1401],
│                   [-1.4916, -0.4511,  0.0633],
│                   [ 0.2273,  0.1149, -1.1401],
│                   [-1.4916, -0.4511,  0.0633],
│                   [ 0.2273,  0.1149, -1.1401],
│                   [-1.4916, -0.4511,  0.0633],
│                   [ 0.2273,  0.1149, -1.1401],
│                   [-1.4916, -0.4511,  0.0633],
│                   [ 0.2273,  0.1149, -1.1401],
│                   [-1.4916, -0.4511,  0.0633],
│                   [ 0.2273,  0.1149, -1.1401]])
└── 'x' --> <FastTreeValue 0x7f7bf35e4f70>
    └── 'c' --> tensor([[ 1.2364, -1.2485,  1.6212, -0.5242],
                        [ 1.2824,  1.5641,  0.6374,  1.2199],
                 

In [23]:
%timeit t_cat(trees)

31.1 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.8 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.4916, -0.4511,  0.0633],
                [ 0.2273,  0.1149, -1.1401]],
       
               [[-1.4916, -0.4511,  0.0633],
                [ 0.2273,  0.1149, -1.1401]],
       
               [[-1.4916, -0.4511,  0.0633],
                [ 0.2273,  0.1149, -1.1401]],
       
               [[-1.4916, -0.4511,  0.0633],
                [ 0.2273,  0.1149, -1.1401]],
       
               [[-1.4916, -0.4511,  0.0633],
                [ 0.2273,  0.1149, -1.1401]],
       
               [[-1.4916, -0.4511,  0.0633],
                [ 0.2273,  0.1149, -1.1401]],
       
               [[-1.4916, -0.4511,  0.0633],
                [ 0.2273,  0.1149, -1.1401]],
       
               [[-1.4916, -0.4511,  0.0633],
                [ 0.2273,  0.1149, -1.1401]]]),
    x: Batch(
           c: tensor([[[ 1.2364, -1.2485,  1.6212, -0.5242],
                       [ 1.2824,  1.5641,  0.6374,  1.2199],
                       [ 0.6198,  0.5993, -0.3168, -1.2198]],
         

In [26]:
%timeit Batch.stack(batches)

79.9 µs ± 329 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.4916, -0.4511,  0.0633],
               [ 0.2273,  0.1149, -1.1401],
               [-1.4916, -0.4511,  0.0633],
               [ 0.2273,  0.1149, -1.1401],
               [-1.4916, -0.4511,  0.0633],
               [ 0.2273,  0.1149, -1.1401],
               [-1.4916, -0.4511,  0.0633],
               [ 0.2273,  0.1149, -1.1401],
               [-1.4916, -0.4511,  0.0633],
               [ 0.2273,  0.1149, -1.1401],
               [-1.4916, -0.4511,  0.0633],
               [ 0.2273,  0.1149, -1.1401],
               [-1.4916, -0.4511,  0.0633],
               [ 0.2273,  0.1149, -1.1401],
               [-1.4916, -0.4511,  0.0633],
               [ 0.2273,  0.1149, -1.1401]]),
    x: Batch(
           c: tensor([[ 1.2364, -1.2485,  1.6212, -0.5242],
                      [ 1.2824,  1.5641,  0.6374,  1.2199],
                      [ 0.6198,  0.5993, -0.3168, -1.2198],
                      [ 1.2364, -1.2485,  1.6212, -0.5242],
                      [ 1.2824,  

In [28]:
%timeit Batch.cat(batches)

151 µs ± 402 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

365 µs ± 6.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
